In [2]:
import os
import sys
import csv
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow_federated as tff
BASE_DIR = ".."
TEST_DIR = "test_dir"
sys.path.insert(0, BASE_DIR)

2024-05-01 01:59:27.790879: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 01:59:27.791003: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 01:59:27.791055: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-01 01:59:27.800374: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 01:59:28.875440: W tensorflow/compiler/

In [31]:
shakespeare_train, shakespeare_test = tff.simulation.datasets.shakespeare.load_data()
fed_datasets = {"train": shakespeare_train, "test": shakespeare_test}
client_sizes = {"train": defaultdict(int), "test": defaultdict(int)}

for split, fed_dataset in fed_datasets.items():
    print(split)
    for client_id in tqdm(fed_dataset.client_ids):
        dataloader = fed_dataset.create_tf_dataset_for_client(client_id)
        for data in dataloader:
            num_chars = len(tf.strings.bytes_split(data['snippets']))
            client_sizes[split][client_id] += num_chars

train


  0%|          | 0/715 [00:00<?, ?it/s]

test


  0%|          | 0/715 [00:00<?, ?it/s]

In [47]:
for split in client_sizes.keys():
    fname = os.path.join(BASE_DIR, 'dataset_statistics', f'shakespeare_client_sizes_{split}.csv')
    pd.DataFrame.from_dict(data=client_sizes[split], orient='index').to_csv(fname, header=True)

## Weird clients that have 0 sizes??

In [58]:
weird_clients = {}
for split in client_sizes.keys():
    weird_clients[split] = [cid for cid in client_sizes[split].keys() if client_sizes[split][cid] == 0]
display(weird_clients)

{'train': [],
 'test': ['ALL_S_WELL_THAT_ENDS_WELL_DROMIO_OF_EPHESUS',
  'ALL_S_WELL_THAT_ENDS_WELL_HERALD',
  'ALL_S_WELL_THAT_ENDS_WELL_OF_EPHESUS',
  'ALL_S_WELL_THAT_ENDS_WELL_PATRICIANS',
  'ALL_S_WELL_THAT_ENDS_WELL_SILIUS',
  'PERICLES__PRINCE_OF_TYRE_CHILDREN',
  'PERICLES__PRINCE_OF_TYRE_GROOM',
  'PERICLES__PRINCE_OF_TYRE_PAGE',
  'PERICLES__PRINCE_OF_TYRE_SERVINGMAN',
  'PERICLES__PRINCE_OF_TYRE_SURREY',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_BOTH',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_ERPINGHAM',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_GARGRAVE',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_HUNTSMAN',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_JUSTICE',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_LAWYER',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_MICHAEL',
  'THE_FIRST_PART_OF_KING_HENRY_THE_FOURTH_SHERIFF',
  'THE_TAMING_OF_THE_SHREW_ALL_SERVANTS',
  'THE_TAMING_OF_THE_SHREW_BANDITTI',
  'THE_TAMING_OF_THE_SHREW_FRANCISCO',
  'THE_TAMING_OF_THE_SHREW_PHILOTUS',


In [59]:
# a random sample
for split in client_sizes.keys():
        dataloader = fed_datasets[split].create_tf_dataset_for_client(fed_dataset.client_ids[123])
        for data in dataloader:
            print(data['snippets'])

tf.Tensor(b'Worth six on him.', shape=(), dtype=string)
tf.Tensor(b"Lepidus is high-colour'd.", shape=(), dtype=string)
tf.Tensor(b"man i' th' world.\nWho, my master?", shape=(), dtype=string)
tf.Tensor(b'Why, then we shall have a stirring world again.', shape=(), dtype=string)
tf.Tensor(b'As they pinch one another by the disposition, he', shape=(), dtype=string)
tf.Tensor(b"Are you so brave? I'll have you talk'd with anon.", shape=(), dtype=string)
tf.Tensor(b'Faith, look you, one cannot tell how to say that;', shape=(), dtype=string)
tf.Tensor(b"Here, sir; I'd have beaten him like a dog, but for", shape=(), dtype=string)
tf.Tensor(b'a ravisher, so it cannot be denied but peace is a great maker of\ncuckolds.\n', shape=(), dtype=string)
tf.Tensor(b"broil'd and eaten him too.\nAnd he's as like to do't as any man I can imagine.", shape=(), dtype=string)
tf.Tensor(b'disturbing the lords within.\nBy my hand, I had thought to have strucken him with', shape=(), dtype=string)
tf.Tensor(b'head

In [60]:
# weird clients
for split in client_sizes.keys():
    for weird_client in weird_clients[split]:
        dataloader = fed_datasets[split].create_tf_dataset_for_client(weird_client)
        for data in dataloader:
            print(data['snippets'])

tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'', shape=(), 

In [62]:
for split in client_sizes.keys():
    for weird_client in weird_clients[split]:
        dataloader = fed_datasets[split].create_tf_dataset_for_client(weird_client)
        for data in dataloader:
            num_chars = len(tf.strings.bytes_split(data['snippets']))
            print(num_chars)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# Just remove them

In [63]:
for split in client_sizes.keys():
    for weird_client in weird_clients[split]:
        del client_sizes[split][weird_client]

In [64]:
weird_clients = {}
for split in client_sizes.keys():
    weird_clients[split] = [cid for cid in client_sizes[split].keys() if client_sizes[split][cid] == 0]
display(weird_clients)

{'train': [], 'test': []}

In [65]:
for split in client_sizes.keys():
    fname = os.path.join(BASE_DIR, 'dataset_statistics', f'shakespeare_client_sizes_{split}.csv')
    pd.DataFrame.from_dict(data=client_sizes[split], orient='index').to_csv(fname, header=True)